<a href="https://colab.research.google.com/github/Abhay27273/Emotional-Intelligent-/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━ 373.2/664.8 MB 41.8 MB/s eta 0:00:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/combined_emotion.csv")  # Update with the correct file path

# Inspect the dataset
print(df.head())
print(df['text'].dtype)

                                                text  emotion
0  say jim how about going for a few beers after ...  neutral
1  can you do pushups of course i can its a piece...  neutral
2  can you study with the radio on no i listen to...  neutral
3  are you all right i will be all right soon i w...  neutral
4  hey john nice skates are they new yeah i just ...  neutral
object


In [ ]:
# Add a task prefix and emotion to the input text
df['input_text'] = "generate empathetic response: " + df['text'] + " [emotion: " + df['emotion'] + "]"
df['target_text'] = df['text']  # Use the same text as the target for now (update this based on your task)

# Inspect the formatted dataset
print(df[['input_text', 'target_text']].head())
print(df['input_text'].dtype)
print(df['target_text'].dtype)

                                          input_text  \
0  generate empathetic response: say jim how abou...   
1  generate empathetic response: can you do pushu...   
2  generate empathetic response: can you study wi...   
3  generate empathetic response: are you all righ...   
4  generate empathetic response: hey john nice sk...   

                                         target_text  
0  say jim how about going for a few beers after ...  
1  can you do pushups of course i can its a piece...  
2  can you study with the radio on no i listen to...  
3  are you all right i will be all right soon i w...  
4  hey john nice skates are they new yeah i just ...  
object
object


In [ ]:
import pandas as pd
from transformers import T5Tokenizer

# Load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Load the dataset
df = pd.read_csv("/content/combined_emotion.csv")  # Update with the correct file path

# Add a task prefix and emotion to the input text
df['input_text'] = "generate empathetic response: " + df['text'].astype(str) + " [emotion: " + df['emotion'].astype(str) + "]"
df['target_text'] = df['text'].astype(str)  # Use the same text as the target for now (update this based on your task)


# Tokenize the dataset
def tokenize_data(df, max_input_length=128, max_target_length=128):
    # Ensure input_text and target_text columns contain only strings
    df['input_text'] = df['input_text'].astype(str)
    df['target_text'] = df['target_text'].astype(str)

    inputs = tokenizer(
        df['input_text'].tolist(),
        max_length=max_input_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    targets = tokenizer(
        df['target_text'].tolist(),
        max_length=max_target_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    return inputs, targets

# Tokenize the dataset
inputs, targets = tokenize_data(df)

# Inspect the tokenized data
print(inputs['input_ids'][0])  # Input token IDs
print(targets['input_ids'][0])  # Target token IDs

tensor([ 3806,     3,    15,    51, 27826,  1773,    10,   497,     3,   354,
          603,   149,    81,   352,    21,     3,     9,   360,    36,   277,
          227,  2634,    25,   214,    24,    19, 24873,    68,    19,   310,
           59,   207,    21,    69,  4639,   125,   103,    25,  1243,    34,
           56,   199,   178,    12,  4839,   103,    25,   310,   317,    78,
            3,    23,  2483,    34,    56,   131,   143,   178,  2886,    11,
         1810, 17056,  1423,   336,    97,     3,    23,  3382,    25,    33,
          269,  2780,   125,  1522,    62,   103,     3,    23,  2483,   473,
          114,  3823,    44,   234,     3,    23,  3130,     3,     9,  1482,
          147,    12,     8,  7868,   213,    62,    54,   577, 10159,     7,
         2444,    11,   942,   128,    13,    69,   803,    24,     7,     3,
            9,   207,   800,     3,    23,  1616,  3157,    63,    11,     3,
            7,  1427,   557,   281,   132,    12,   577,     1])

In [ ]:
from torch.utils.data import Dataset, random_split # Import random_split

class T5Dataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs['input_ids'][idx],
            'attention_mask': self.inputs['attention_mask'][idx],
            'labels': self.targets['input_ids'][idx]
        }

    def __len__(self):
        return len(self.inputs['input_ids'])

# Create the dataset
dataset = T5Dataset(inputs, targets)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size]) # Use random_split directly

In [ ]:
import torch
from transformers import T5ForConditionalGeneration

# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Move the model to GPU (if available)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch in train_loader:
        # Move batch to GPU (if available)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Print average loss for the epoch
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss}")

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [ ]:
model.eval()  # Set the model to evaluation mode
total_loss = 0

with torch.no_grad():
    for batch in val_loader:
        # Move batch to GPU (if available)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

# Print validation loss
avg_loss = total_loss / len(val_loader)
print(f"Validation Loss: {avg_loss}")

In [ ]:
# Save the model
model.save_pretrained('t5_finetuned')
tokenizer.save_pretrained('t5_finetuned')

In [ ]:
# Load the fine-tuned model
model = T5ForConditionalGeneration.from_pretrained('t5_finetuned')
tokenizer = T5Tokenizer.from_pretrained('t5_finetuned')

# Move the model to GPU (if available)
model.to(device)

# Generate text
input_text = "generate empathetic response: I feel so happy today! [emotion: happiness]"
input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(device)

# Generate output
outputs = model.generate(input_ids, max_length=50)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)